In [6]:
import sys
import os

# Add the path to the directory containing inference.py
sys.path.append(r"C:\Users\mahmo\OneDrive\Desktop\Spring 2025\MLops notes\Final_Project\Final_Project\src")

import pandas as pd
import hopsworks
from datetime import timedelta
import config  # or from config import YOUR_VARIABLE if needed

# Import available functions from inference.py
from inference import get_feature_store, load_model_from_registry, get_model_predictions


In [14]:
def fetch_hourly_rides(hours_back=12):
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=config.FEATURE_GROUP_VERSION
    )

    # Read entire feature group (required in Python)
    df_all = fg.read()

    # Now get latest timestamp manually
    latest_hour = pd.to_datetime(df_all["pickup_hour"].max(), utc=True)
    current_hour = (latest_hour - timedelta(hours=hours_back)).floor('h')

    # Filter using a query
    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    df_result = query.read()
    print(f"✅ Found {len(df_result)} rows from {current_hour} to {latest_hour}")
    return df_result

df_actual = fetch_hourly_rides(hours_back=12)
df_actual.head()


2025-05-10 07:43:55,198 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 07:43:55,207 INFO: Initializing external client
2025-05-10 07:43:55,210 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 07:43:56,020 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.22s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 
✅ Found 108 rows from 2023-12-31 11:00:00+00:00 to 2023-12-31 23:00:00+00:00


,pickup_hour,pickup_location_id,rides
0,2023-12-31 14:00:00+00:00,6948.10,25
1,2023-12-31 17:00:00+00:00,6948.10,26
2,2023-12-31 09:00:00+00:00,5329.03,2
3,2023-12-31 08:00:00+00:00,5329.03,3
4,2023-12-31 13:00:00+00:00,6948.10,17


In [18]:
def fetch_recent_predictions(hours_back=12):
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,  # "citi_bike_prediction"
        version=2
    )

    # Read to find latest available prediction hour
    df_all = fg.read()
    latest_hour = pd.to_datetime(df_all["pickup_hour"].max(), utc=True)
    current_hour = (latest_hour - timedelta(hours=hours_back)).floor('h')

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    df_result = query.read()
    print(f"✅ Retrieved {len(df_result)} rows from {current_hour} to {latest_hour}")
    return df_result

df_pred = fetch_recent_predictions(12)
df_pred.head()


2025-05-10 07:50:22,099 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 07:50:22,106 INFO: Initializing external client
2025-05-10 07:50:22,106 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 07:50:22,905 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 
✅ Retrieved 54 rows from 2023-12-31 11:00:00+00:00 to 2023-12-31 23:00:00+00:00


,pickup_location_id,pickup_hour,predicted_rides
0,6140.05,2023-12-31 08:00:00+00:00,8
1,6140.05,2023-12-31 06:00:00+00:00,1
2,6140.05,2023-12-31 11:00:00+00:00,25
3,6948.10,2023-12-31 14:00:00+00:00,20
4,6948.10,2023-12-31 21:00:00+00:00,7


In [21]:
merged_df = pd.merge(
    df_actual,
    df_pred,
    on=['pickup_location_id', 'pickup_hour']
)

merged_df['absolute_error'] = abs(merged_df['predicted_rides'] - merged_df['rides'])
merged_df.head()


,pickup_hour,pickup_location_id,rides,predicted_rides,absolute_error
0,2023-12-31 14:00:00+00:00,6948.10,25,20,5
1,2023-12-31 17:00:00+00:00,6948.10,26,23,3
2,2023-12-31 09:00:00+00:00,5329.03,2,5,3
3,2023-12-31 08:00:00+00:00,5329.03,3,3,0
4,2023-12-31 13:00:00+00:00,6948.10,17,21,4


In [22]:
mae_by_hour = (
    merged_df
    .groupby('pickup_hour')['absolute_error']
    .mean()
    .reset_index()
    .rename(columns={'absolute_error': 'MAE'})
)
mae_by_hour.head()


,pickup_hour,MAE
0,2023-12-31 06:00:00+00:00,1.000000
1,2023-12-31 07:00:00+00:00,1.666667
2,2023-12-31 08:00:00+00:00,1.000000
3,2023-12-31 09:00:00+00:00,1.666667
4,2023-12-31 10:00:00+00:00,5.000000


In [23]:
import plotly.express as px

fig = px.line(
    mae_by_hour,
    x='pickup_hour',
    y='MAE',
    title='Mean Absolute Error (MAE) by Pickup Hour',
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},
    markers=True
)

fig.show()


In [24]:
print("🔢 Average MAE across hours:", mae_by_hour["MAE"].mean())


🔢 Average MAE across hours: 5.222222222222222


In [25]:
print("🔍 FG name:", config.FEATURE_GROUP_MODEL_PREDICTION)
print("🔍 FG version:", config.FEATURE_GROUP_MODEL_PREDICTION_VERSION)


🔍 FG name: citi_bike_prediction
🔍 FG version: 2
